In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.032962,0.136763,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.032200,0.131800,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.031425,0.128250,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.030650,0.124700,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029875,0.121150,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 28 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_er"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.94227752263
CV Scores:  0.951939361609, 0.907887601382, 0.939449135649, 0.950728465425, 0.937319953228, 0.961075347668, 0.954862959088, 0.94766646795, 0.92317434232, 0.948671591977
OOB score: 0.943608106988
Feature Importances:
('wp_LST.night', 0.26224430199444726)
('wp_LST.day', 0.15360421655925366)
('air_temp', 0.1453392802486449)
('PET', 0.11808688584316457)
('wind_speed', 0.079524738461726974)
('sw_in', 0.069084755902904277)
('VPD', 0.047278114338017402)
('wp_ndvi', 0.034736207566437152)
('wp_lswi2', 0.033031491553853674)
('wp_evi', 0.030815928217107033)
('LW_IN.wp', 0.025063866150946225)
('precip', 0.0011902131634969626)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.944514278318
CV Scores:  0.956423469488, 0.912004330869, 0.93504832728, 0.940638495042, 0.955028481834, 0.962300505058, 0.956637288762, 0.944716177572, 0.928530547733, 0.953815159541
Feature Importances:
('wp_lswi2', 0.14079928613049617)
('wp_ndvi', 0.13486530190608301)
('wp_evi', 0.12557257646551298)
('wp_LST.night', 0.12556529875765565)
('wind_speed', 0.10114832371981006)
('wp_LST.day', 0.088860886816798434)
('sw_in', 0.067330169894142464)
('VPD', 0.061247829552346637)
('air_temp', 0.055052309505242265)
('PET', 0.039733714970871518)
('LW_IN.wp', 0.036167880401619251)
('precip', 0.023656421879421609)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.86846683826
CV Scores:  0.889780554708, 0.82532339553, 0.819853971042, 0.870783503928, 0.85814133888, 0.918614582952, 0.901387017421, 0.903123656251, 0.820893546629, 0.876766815257


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 762.12671
Step #501, epoch #50, avg. loss: 20.44991
Step #1001, epoch #100, avg. loss: 5.50194
Step #1501, epoch #150, avg. loss: 3.63673
Step #2001, epoch #200, avg. loss: 2.79693
Step #2501, epoch #250, avg. loss: 2.32774
Step #3001, epoch #300, avg. loss: 2.06789
Step #3501, epoch #350, avg. loss: 1.85985
Step #4001, epoch #400, avg. loss: 1.74393
Step #4501, epoch #450, avg. loss: 1.66846
Step #1, avg. loss: 838.61639
Step #501, epoch #50, avg. loss: 20.75419
Step #1001, epoch #100, avg. loss: 5.37160
Step #1501, epoch #150, avg. loss: 3.60897
Step #2001, epoch #200, avg. loss: 2.81992
Step #2501, epoch #250, avg. loss: 2.35575
Step #3001, epoch #300, avg. loss: 2.01378
Step #3501, epoch #350, avg. loss: 1.81269
Step #4001, epoch #400, avg. loss: 1.69704
Step #4501, epoch #450, avg. loss: 1.57168
Step #1, avg. loss: 761.08862
Step #501, epoch #50, avg. loss: 20.73128
Step #1001, epoch #100, avg. loss: 5.27556
Step #1501

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x1159b5488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)